The data:
112 stocks book samples over 10 min time buckets
112 stocks trade data over 10 min time buckets

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
files=[]
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))


In [ ]:
s='86'
sf=[]
sf=[file for file in files if str('=' + s) in file]

In [ ]:
a=pd.read_parquet(sf[0])
b=pd.read_parquet(sf[1])
a.info(),b.info()

In [ ]:
b.head()

In [ ]:
b[['seconds_in_bucket','time_id']].hist()

In [ ]:
a.info()

In [ ]:
b.ask_price1.hist(bins=200)
b.bid_price1.hist(bins=200)

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
import glob
import seaborn as sns
from sklearn.model_selection import train_test_split as split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score


list_order_book_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*')
list_trade_file_train = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/*')
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')

def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
def rmspe(y, y_pred):
    return  (np.sqrt(np.mean(np.square((y - y_pred) / y))))

# Feature creation
This code creates raw features based on the raw data:
from the book data, per stock and time bucket:
- realized volatility
- sum of log returns
- sum of bid and ask sizes for each level
- mean spread

from the trade data, per stock and time bucket:
- trade size
- order count

*trade price is left out since it is accaunted for by 'wap'

In [ ]:
def book_features(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_book_data['spread'] = df_book_data['ask_price1'] - df_book_data['bid_price1']
    df_book_features =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_book_features = df_book_features.rename(columns = {'log_return':prediction_column_name})
    df_agg_data = pd.DataFrame(df_book_data.groupby(['time_id'])[['log_return','bid_size1','ask_size1','bid_size2','ask_size2']].sum()).reset_index()
    stock_id = file_path.split('=')[1]
    df_book_features['stock_id'] = stock_id
    df_agg_data = df_agg_data.rename(columns = {'log_return':'sum_log_return'})
    df_agg_data['row_id'] = df_agg_data['time_id'].apply(lambda x:f'{stock_id}-{x}')
    df_spread = pd.DataFrame(df_book_data.groupby(['time_id'])['spread'].mean()).reset_index()
    df_spread['row_id'] = df_spread['time_id'].apply(lambda x:f'{stock_id}-{x}')                        
    df_book_features = df_book_features.merge(df_agg_data)
    df_book_features = df_book_features.merge(df_spread)                         
    return df_book_features

df_book_features = pd.DataFrame()
for file in list_order_book_file_train:
    a=book_features(file,prediction_column_name='realized_vol')
    df_book_features=pd.concat([df_book_features,a])

**The resulting book features df:**

In [ ]:
df_book_features.info()

In [ ]:
def trade_features(file_path):
    df_trade_data = pd.read_parquet(file_path)
    df_trade_features =  pd.DataFrame(df_trade_data.groupby(['time_id'])[['size','order_count']].sum()).reset_index()
    df_trade_features = df_trade_features.rename(columns = {'size':'trade_size'})
    stock_id = file_path.split('=')[1]
    df_trade_features['stock_id'] = stock_id
    df_trade_features['row_id'] = df_trade_features['time_id'].apply(lambda x:f'{stock_id}-{x}')                                        
    return df_trade_features

df_trade_features = pd.DataFrame()
for file in list_trade_file_train:
    a=trade_features(file)
    df_trade_features=pd.concat([df_trade_features,a])

**the resulting trade data features df:**

In [ ]:
df_trade_features.info()

In [ ]:
df_raw_features = df_book_features.merge(df_trade_features,how='left')

In [ ]:
df_raw_features.info()

In [ ]:
df_trade_features.stock_id.nunique()

# Feature engeneering

First we will calculate the future realized volatility for each time bucket and stock (this is the target).

by doing that we have to depart with all the last time buckets - it won't have future data.

In [ ]:
df_raw_features['future_vol'] = df_raw_features.groupby(['stock_id'])['realized_vol'].shift(-1)
df_raw_features = df_raw_features[~df_raw_features['future_vol'].isnull()]

The bid and ask sizes will be used to claculate the book depth:
- the ratio between first level bid and ask i.e. 'baratio'
- the ratio between the bid size and the average time bucket trade size i.e. 'bratio'
- the ratio between the ask size and the average time bucket trade size i.e. 'aratio'
- the average order

In [ ]:
df_raw_features.groupby(['stock_id','time_id'])['trade_size'].sum().unstack().mean(axis=1)

In [ ]:
df_raw_features['baratio'] = df_raw_features['bid_size1'] / df_raw_features['ask_size1']
sizes = df_raw_features.groupby(['stock_id','time_id'])['trade_size'].sum().unstack().mean(axis=1)
df_raw_features = df_raw_features.merge(sizes.rename('mean_trade_size'), left_on='stock_id', right_on='stock_id')
df_raw_features['bratio']=(df_raw_features['bid_size1'] + df_raw_features['bid_size2'])/df_raw_features['mean_trade_size']
df_raw_features['aratio']=(df_raw_features['ask_size1'] + df_raw_features['ask_size2'])/df_raw_features['mean_trade_size']
df_raw_features['mean_order']=df_raw_features['trade_size']/df_raw_features['order_count']
df_raw_features.head()

In [ ]:
# df_raw_features['bratio']=(df_raw_features['bid_size1'] + df_raw_features['bid_size2'])/df_raw_features['mean_trade_size']
# df_raw_features['aratio']=(df_raw_features['ask_size1'] + df_raw_features['ask_size2'])/df_raw_features['mean_trade_size']
df_raw_features.head()

- most of the features seem to distribute log normaly. so in order for them to fit the scale of our target - I will log them

Transformations

In [ ]:
logged = df_raw_features[['spread','baratio','bratio','aratio','mean_order']]
logged = np.log1p(logged)
#hundreds = df_raw_features[['realized_vol','future_vol']]*100

# EDA
now we will see some analysis of the calculated features

In [ ]:
df_final_features = df_raw_features.copy()[['time_id','stock_id','row_id','realized_vol','future_vol', 'sum_log_return']]
df_final_features = df_final_features.merge(logged,left_index=True, right_index=True)
#df_final_features = df_final_features.merge(hundreds,left_index=True, right_index=True)
df_final_features.head()

# Removing outliars

In [ ]:
df_final_features = df_final_features[df_final_features['future_vol']<0.06]
df_final_features = df_final_features[df_final_features['realized_vol']<0.06]
df_final_features = df_final_features[df_final_features['spread']<0.012]

In [ ]:
len(df_final_features)

In [ ]:
df_final_features.hist(bins=80,figsize=(9,6))

- on most cases the features seem to distribute log normaly (except for the returns which are allready logged)

In [ ]:
sns.heatmap(df_final_features.corr().abs())
df_final_features.corr()

In [ ]:
#sns.pairplot(df_final_features.iloc[:,2:], height=1.5)
df_final_features.plot.scatter('realized_vol','spread')
df_final_features.plot.scatter('realized_vol','future_vol')
df_final_features.plot.scatter('aratio','bratio')
df_final_features.columns

- the spread is heavily correlated with the realized volatility
- the ask ratio and the bid ratio are correlated, but the scatter isn't so clear. 
remove one? log them?
- the other features are not very correlated to each other and to the target. 
- question - the outliers indicate special trade points. do we leave them? seperate?

In [ ]:
df_final_features.head()

# Linear Regression
- aratio and bratio are correleted. so for the linear regression I'll use only one 

In [ ]:
#df_final_features.columns
X = df_final_features[['realized_vol','sum_log_return','spread','baratio','bratio']]
y = df_final_features['future_vol']
lin_model_1 = LinearRegression(fit_intercept=False).fit(X, y)

In [ ]:
list(zip(X.columns, lin_model_1.coef_))

In [ ]:
y_pred = lin_model_1.predict(X)
ax = sns.scatterplot(x=y, y=y_pred)
ax.plot(y, y, 'r')

In [ ]:
mse(y, y_pred, squared=False)

In [ ]:
R2 = round(r2_score(y_true = y, y_pred = y_pred),3)
RMSPE = round(rmspe(y = y, y_pred = y_pred),3)
print(f'Performance of the linear prediction: R2 score: {R2}, RMSPE: {RMSPE}')

# Decision Tree

In [ ]:
X = df_final_features[['realized_vol','sum_log_return','spread','baratio','bratio','aratio']]
y = df_final_features['future_vol']
dt_model_1 = DecisionTreeRegressor(max_leaf_nodes=1000).fit(X, y)

In [ ]:
y_pred = dt_model_1.predict(X)
ax = sns.scatterplot(x=y, y=y_pred)
ax.plot(y, y, 'r')

In [ ]:
pd.Series(dt_model_1.feature_importances_,
          index=X.columns).sort_values(ascending=False).plot.barh()

In [ ]:
msle(y, y_pred)**0.5

In [ ]:
R2 = round(r2_score(y_true = y, y_pred = y_pred),3)
RMSPE = round(rmspe(y = y, y_pred = y_pred),3)
print(f'Performance of DT model: R2 score: {R2}, RMSPE: {RMSPE}')

# KNN

In [ ]:
knn_model_1 = KNeighborsRegressor(n_neighbors=4).fit(X, y)
y_pred = knn_model_1.predict(X)

In [ ]:
ax = sns.scatterplot(x=y, y=y_pred)
ax.plot(y, y, 'r')

In [ ]:
R2 = round(r2_score(y_true = y, y_pred = y_pred),3)
RMSPE = round(rmspe(y = y, y_pred = y_pred),3)
print(f'Performance of KNN: R2 score: {R2}, RMSPE: {RMSPE}')

# Validating the Decision tree

> To be continued with better models